In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
from PIL import ImageTk
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision import models, transforms
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
normal_path = "/img/brain_data/Normal/"
stroke_path = "/img/brain_data/Stroke/"
normal_folder = os.listdir(normal_path)
stroke_folder = os.listdir(stroke_path)

print("Images in Normal data:", len(normal_folder))
print("Images in Stroke data:", len(stroke_folder))

data=[]

for img_file in normal_folder:
    image = Image.open(normal_path + img_file)
    image = image.resize((229, 229))
    image = image.convert('RGB')
    data.append(np.array(image))

for img_file in stroke_folder:
    image = Image.open(stroke_path + img_file)
    image= image.resize((229, 229))
    image = image.convert('RGB')
    data.append(np.array(image))

normal_label =[0] * len(normal_folder)
stroke_label = [1] * len(stroke_folder)
Target_label = normal_label + stroke_label

x = np.array(data)
y = np.array(Target_label)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, shuffle=True)

x_train_s = x_train / 255.0
x_test_s = x_test / 255.0

Images in Normal data: 950
Images in Stroke data: 950


In [ ]:
#Convert to torch tensors
x_train_tensor = torch.tensor(x_train_s).permute(0, 3, 1, 2).float()

#Convert (batch, height, width, channels) to (batch, channels, height, width)
x_test_tensor = torch.tensor(x_test_s).permute(0, 3, 1, 2).float()

y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

#Create DataLoader for training and testing
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#Feature extraction
def extract_features(model, dataloader, device):
    model.eval()  #Set to evaluation mode
    features = []
    labels = []

    with torch.no_grad():
        for inputs, target in dataloader:
            inputs, target = inputs.to(device), target.to(device)
            output = model(inputs)  #Forward pass to get features

            # Collect features
            features.append(output.cpu().numpy())  #Append the output (features)
            labels.append(target.cpu().numpy())  #Collecting actual labels

    return np.concatenate(features), np.concatenate(labels)

#Load model for feature extraction and moving to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models_dict = {
    #'AlexNet': models.alexnet(pretrained=True),
    #'VGG-19': models.vgg19(pretrained=True),
    'InceptionV3': models.inception_v3(pretrained=True),
    #'ShuffleNet': models.shufflenet_v2_x1_0(pretrained=True),
    #'NasNet': timm.create_model('nasnetalarge', pretrained=True)
}

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 58.1MB/s]


In [ ]:
#Remove the final classification layer to get feature vectors
for model in models_dict.values():

    # Remove the last layer
    if isinstance(model, models.Inception3):
        model.AuxLogits = nn.Sequential()  #Remove the auxiliary classifier
        model.fc = nn.Sequential()  #Remove the fully connected layers

    #GPU/CPU
    model.to(device)

features_dict = {} #modeltraining
labels_dict = {}

for model_name, model in models_dict.items():
    print(f"Extracting features using {model_name}...")
    features, labels = extract_features(model, test_loader, device)
    features_dict[model_name] = features
    labels_dict[model_name] = labels


Extracting features using InceptionV3...


In [ ]:
INC =  features_dict['InceptionV3']
print(INC)

[[0.46653566 0.2511339  0.16805016 ... 0.28949544 0.7967827  0.7536384 ]
 [0.8116022  2.2404943  0.47563195 ... 1.2850885  1.1036029  0.02601482]
 [0.5054274  0.09964101 0.13118584 ... 0.39926076 0.330751   0.03566289]
 ...
 [0.49979016 0.10419016 1.058598   ... 0.5111747  0.37345746 0.25737903]
 [0.74136704 0.20168933 0.5377217  ... 0.58027536 0.7138277  0.7707926 ]
 [0.52496636 0.8385651  0.45286474 ... 1.5172292  1.209733   0.14549237]]


In [ ]:
!pip install DEAP

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.6 MB/s eta 0:00:00


In [ ]:
#Genetic Algorithm for feature selection

from deap import base, creator, tools, algorithms
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

#Evaluating fitness of a chromosome
def evaluate(individual, features, labels):
    #Use the binary chromosome (0 or 1) to select features
    selected_features = features[:, np.array(individual, dtype=bool)]
    if selected_features.shape[1] == 0:  # Avoid/handle empty feature sets
        return 0.01                      # Return a small non-zero value instead of 0

    #Train an SVM classifier and calculate accuracy using cross-validation
    clf = SVC(kernel="linear", random_state=42)
    scores = cross_val_score(clf, selected_features, labels, cv=5)  #Accuracy is calculated using 5-fold cross-validation
    return np.mean(scores),

#DEAP framework
num_features = features_dict['InceptionV3'].shape[1]  # Number of features

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", lambda: np.random.randint(0, 2))  #Binary genes
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=num_features)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)                      #Crossover operation
toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)        #Mutation operation
toolbox.register("select", tools.selTournament, tournsize=3)    #Selection operation
toolbox.register("evaluate", evaluate, features=features_dict['InceptionV3'], labels=labels_dict['InceptionV3'])

#Genetic Algorithm Parameters
population_size = 50
generations = 30
cxpb = 0.5  #Crossover probability
mutpb = 0.2  #Mutation probability

#Running Genetic Algorithm
population = toolbox.population(n=population_size)
hall_of_fame = tools.HallOfFame(1)  #best solution
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

print("Running Genetic Algorithm for Feature Selection...")
final_population, logbook = algorithms.eaSimple(
    population, toolbox, cxpb, mutpb, generations, stats, hall_of_fame, verbose=True
)

# Extract Best Features
best_individual = hall_of_fame[0]
selected_features_idx = np.array(best_individual, dtype=bool)

print(f"Selected {np.sum(selected_features_idx)} features out of {num_features}.")
selected_features = features_dict['InceptionV3'][:, selected_features_idx]


#clf = SVC(kernel="linear",C=0.1,random_state=42)
#clf.fit(selected_features, labels_dict['InceptionV3'])
#accuracy = clf.score(selected_features, labels_dict['InceptionV3'])

# Now apply the selected feature mask on original (full) data
full_features = features_dict['InceptionV3'][:, selected_features_idx]
full_labels = labels_dict['InceptionV3']

# Split into train/test again (this time for classifier training/testing)
X_train_sel, X_test_sel, y_train_sel, y_test_sel = train_test_split(
    full_features, full_labels, test_size=0.3, random_state=42, shuffle=True
)

# Train classifier on selected features (training set only)
clf = SVC(kernel="linear", C=0.1, random_state=42)
clf.fit(X_train_sel, y_train_sel)

# Test on test set
y_pred = clf.predict(X_test_sel)
accuracy = accuracy_score(y_test_sel, y_pred)

print(f"Classifier Test Accuracy using GA-selected features: {accuracy:.2f}")



Running Genetic Algorithm for Feature Selection...
gen	nevals	avg     	min     	max     
0  	50    	0.638386	0.587719	0.673684
1  	29    	0.649825	0.607018	0.678947
2  	37    	0.659053	0.638596	0.682456
3  	32    	0.662632	0.629825	0.684211
4  	34    	0.668035	0.636842	0.698246
5  	25    	0.672947	0.638596	0.705263
6  	28    	0.680561	0.647368	0.705263
7  	32    	0.688842	0.668421	0.708772
8  	29    	0.691614	0.649123	0.708772
9  	33    	0.697614	0.663158	0.708772
10 	31    	0.699193	0.677193	0.708772
11 	30    	0.701509	0.680702	0.708772
12 	30    	0.699965	0.663158	0.714035
13 	31    	0.701439	0.67193 	0.714035
14 	32    	0.702877	0.682456	0.710526
15 	35    	0.702   	0.670175	0.719298
16 	35    	0.704035	0.663158	0.719298
17 	38    	0.705158	0.678947	0.722807
18 	24    	0.709544	0.687719	0.722807
19 	34    	0.710386	0.691228	0.722807
20 	31    	0.714316	0.7     	0.729825
21 	27    	0.718667	0.705263	0.729825
22 	31    	0.720632	0.707018	0.729825
23 	29    	0.720281	0.710526	0.729825

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

#Defining BiLSTM Model

class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  #Multiply by 2 for bidirectional output
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  #LSTM output
        out = self.fc(lstm_out[:, -1, :])  #Take the last time step
        return self.softmax(out)

# Define Model Parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = selected_features.shape[1]     #Feature vector size from InceptionV3
hidden_size = 128                           #LSTM hidden layer size
num_layers = 2                              #Number of LSTM layers
num_classes = 2                             #Binary classification (Normal vs. Stroke)

#Initialize the BiLSTM Model
bilstm_model = BiLSTM(input_size, hidden_size, num_layers, num_classes).to(device)

#Define Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(bilstm_model.parameters(), lr=0.001)

print(bilstm_model)


BiLSTM(
  (lstm): LSTM(1034, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#Ensure selected_features is being used (not raw images)
X_train_tensor = torch.tensor(selected_features, dtype=torch.float32)
y_train_tensor = torch.tensor(labels_dict['InceptionV3'], dtype=torch.long)

#Ensure the input shape matches the LSTM expectation
print(f"Expected input size: {bilstm_model.lstm.input_size}")
print(f"Actual feature vector size: {X_train_tensor.shape[1]}")  #Check shape

#DataLoader setup
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bilstm_model.to(device)

#Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(bilstm_model.parameters(), lr=0.001)

#Training Loop
num_epochs = 10

for epoch in range(num_epochs):
    bilstm_model.train()

    total_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        # Ensure proper input shape: (batch_size, seq_length=1, input_size)
        x_batch = x_batch.view(x_batch.size(0), 1, -1)  # Ensure shape (B, 1, feature_dim)

        #CrossEntropy Loss is used for classification.
        #Adam Optimizer adjusts model weights.
        #Gradient Descent: Backpropagation updates weights.

        # Forward pass
        outputs = bilstm_model(x_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("Training complete!")



Expected input size: 1034
Actual feature vector size: 1034
Epoch [1/10], Loss: 0.6922
Epoch [2/10], Loss: 0.6677
Epoch [3/10], Loss: 0.6000
Epoch [4/10], Loss: 0.5468
Epoch [5/10], Loss: 0.4801
Epoch [6/10], Loss: 0.4470
Epoch [7/10], Loss: 0.4105
Epoch [8/10], Loss: 0.3887
Epoch [9/10], Loss: 0.3705
Epoch [10/10], Loss: 0.3570
Training complete!


In [ ]:
import torch
import numpy as np
from PIL import Image


def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.resize((224, 224))
    image = image.convert('RGB')
    image = np.array(image) / 255.0  # Normalize
    image_tensor = torch.tensor(image).permute(2, 0, 1).unsqueeze(0).float().to(device)
    return image_tensor  # Fixed return statement

#Extract Features for a Single Image
def extract_features_single_image(image_path, model):
    model.eval()
    with torch.no_grad():
        image_tensor = preprocess_image(image_path)
        features = model(image_tensor)  #Extract features
    return features.cpu().numpy().flatten()

#Predict Stroke vs. Normal
def predict(image_path, inception_model, bilstm_model):
    features = extract_features_single_image(image_path, inception_model)
    features_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0).unsqueeze(1).to(device)  # Reshape for LSTM

    bilstm_model.eval()
    with torch.no_grad():
        output = bilstm_model(features_tensor)

    predicted_class = torch.argmax(output, dim=1).item()
    return "Stroke" if predicted_class == 1 else "Normal"



In [ ]:
#Convert extracted features into PyTorch tensors
test_features_tensor = torch.tensor(selected_features, dtype=torch.float32)
test_labels_tensor = torch.tensor(labels_dict['InceptionV3'], dtype=torch.long)

#Create DataLoader using extracted features instead of images
test_dataset = TensorDataset(test_features_tensor, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#evaluating model performances

def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            #Ensure correct shape for BiLSTM: (batch_size, seq_length=1, input_size)
            inputs = inputs.view(inputs.size(0), 1, -1)  # -1 auto-fits feature vector size (1033)

            #Forward pass
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    #Compute evaluation metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average="binary")
    recall = recall_score(all_labels, all_preds, average="binary")
    f1 = f1_score(all_labels, all_preds, average="binary")

    print(f" Model Evaluation Results:")
    print(f"Accuracy: {accuracy:.4f}")      #Overall correctness.
    print(f"Precision: {precision:.4f}")    #How many detected strokes are actually strokes.
    print(f"Recall: {recall:.4f}")          #Overall correctness.
    print(f"F1 Score: {f1:.4f}")            #Harmonic mean of Precision & Recall.

    return accuracy, precision, recall, f1

# Run evaluation on test set
evaluate_model(bilstm_model, test_loader, device)
torch.save(bilstm_model.state_dict(), "bilstm_model.pth")


 Model Evaluation Results:
Accuracy: 0.9316
Precision: 0.9640
Recall: 0.9024
F1 Score: 0.9322


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from torchvision import models

# Load trained BiLSTM model
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes): # Corrected the method name to __init__
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Load the saved model
input_size = selected_features.shape[1]
hidden_size = 128
num_layers = 2
num_classes = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bilstm_model = BiLSTM(input_size, hidden_size, num_layers, num_classes).to(device)
bilstm_model.load_state_dict(torch.load("bilstm_model.pth", map_location=device))
bilstm_model.eval()

# Load InceptionV3 for feature extraction
inception = models.inception_v3(pretrained=True)
inception.fc = nn.Identity()
inception.to(device)
inception.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((229, 229)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

def is_ct_scan(image: Image.Image) -> bool:
    return np.mean(np.array(image)) < 200  # CT scans are usually grayscale & dark

def predict(image):
    if not is_ct_scan(image):
        return "Error: Please upload a valid CT scan image."

    # Preprocess the image
    image = transform(image).unsqueeze(0).to(device)  # Shape: (1, 3, 299, 299)

    # Extract Features using InceptionV3
    with torch.no_grad():
        features = inception(image).cpu().numpy()

    print(f"🟢 Extracted feature shape: {features.shape}")  # Should be (1, feature_dim)

    # If you have selected features, apply indexing
    if 'selected_features_idx' in globals():
        features = features[:, selected_features_idx]  # Ensure this works correctly

    # Convert to tensor & reshape for LSTM
    features_tensor = torch.tensor(features, dtype=torch.float32).to(device)

    # ✅ **Fix the Reshaping Issue**
    features_tensor = features_tensor.unsqueeze(0)  # (1, feature_dim) -> (1, 1, feature_dim)

    print(f"🟢 Final input shape to LSTM: {features_tensor.shape}")  # Should be (1, 1, feature_dim)

    # Pass through BiLSTM model
    with torch.no_grad():
        output = bilstm_model(features_tensor)
        prediction = torch.argmax(output, dim=1).item()

    return "Stroke" if prediction == 1 else "Normal"


# Gradio Interface
demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type="pil"),
                    outputs=gr.Textbox(),
                    title="Stroke Detection System",
                    description="Upload a brain CT scan image to detect stroke.")

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cf698ec89b54dd42c8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
